<a href="https://colab.research.google.com/github/mohamedtal/PFE2021/blob/main/04_04_2021_task1_b_MultiChannel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import os
import random
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.python.keras import Model
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras import layers
from keras.applications.resnet50 import ResNet50
from keras.applications import DenseNet121
from keras.utils import Sequence
from keras.layers import Dense, Dropout, Flatten, Input, ZeroPadding2D, GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras.models import Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras import optimizers
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import tensorflow as tf


from pickle import load
from numpy import array
from keras.utils.vis_utils import plot_model
from keras.layers import Embedding
from keras.layers.merge import concatenate
from tensorflow.keras.layers import AvgPool2D, GlobalAveragePooling2D, MaxPool2D
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dense
from tensorflow.keras.layers import AvgPool2D, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ReLU, concatenate
import tensorflow.keras.backend as K


In [ ]:
# before all don't forget to use the GPU

# at first we need to get the dataset from the drive 
# pfe2021.1@gmail.com		zakimoha123
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
#unzip the dataset

# for the train
!unzip /content/drive/MyDrive/dataset/task1-batches/batchesTrain.zip
!unzip /content/drive/MyDrive/dataset/task1-batches/batchesTest.zip

# read data

In [ ]:
# read images 
path_train = 'batchesTrain/'
list_batches = []
for i in range(1,17):
  path_batch = path_train+str(i)
  batch_images = []
  for c in range(5):
    path_class = path_batch+'/'+str(c)
    # get the list of images in each class
    list_images =  os.listdir(path_class)
    i = 0
    for image in list_images :
      path_image = path_class+'/'+image
      img = cv2.imread(path_image)
      img = cv2.resize(img,(224,224))
      batch_images.append(img)
      if i>=300:
        break
      i = i+1
  list_batches.append(np.array(batch_images))

In [ ]:
# read labels
path = 'batchesTrain/1/'
trainY = []
for c in range(5):
  y = np.zeros(5,dtype=int)
  y[c] = 1
  path_class = path+str(c)
  # get the list of images in each class
  list_images =  os.listdir(path_class)
  i = 0
  for image in list_images :
    trainY.append(y)
    if i>=300:
        break
    i = i+1
trainY = np.array(trainY)
# convert to numpy array

In [ ]:
# read images 
path_test = 'batchesTest/'
list_batches_test = []
for i in range(1,17):
  path_batch = path_test+str(i)
  batch_images = []
  for c in range(5):
    path_class = path_batch+'/'+str(c)
    # get the list of images in each class
    list_images =  os.listdir(path_class)
    for image in list_images :
      path_image = path_class+'/'+image
      img = cv2.imread(path_image)
      img = cv2.resize(img,(224,224))
      batch_images.append(img)
  list_batches_test.append(np.array(batch_images))

In [ ]:
# read labels
path = 'batchesTest/1/'
trainY_test = []
for c in range(5):
  y = np.zeros(5,dtype=int)
  y[c] = 1
  path_class = path+str(c)
  # get the list of images in each class
  list_images =  os.listdir(path_class)
  for image in list_images :
    trainY_test.append(y)
trainY_test = np.array(trainY_test)
# convert to numpy array

In [ ]:
print(trainY_test.shape)

(500, 5)


# data preprocessing



In [ ]:
def preprocessFunction(image):
    #image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    kopya = image.copy()
    kopya = cv2.cvtColor(kopya,cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(kopya,(5,5),0)
    thresh = cv2.threshold(blur,10,255,cv2.THRESH_BINARY)[1]
    kontur = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    kontur = kontur[0][0]
    kontur = kontur[:,0,:]
    x1 = tuple(kontur[kontur[:,0].argmin()])[0]
    x2 = tuple(kontur[kontur[:,0].argmax()])[0]
    y1 = tuple(kontur[kontur[:,1].argmin()])[1]
    y2 = tuple(kontur[kontur[:,1].argmax()])[1]
    x = int(x2-x1)*4//50
    y = int(y2-y1)*5//50
    kopya2 = image.copy()
    if x2-x1>100 and y2-y1>100 :
        kopya2 = kopya2[y1+y:y2-y , x1+x:x2-x]
        kopya2 = cv2.resize(kopya2,(1024,1024))
    lab = cv2.cvtColor(kopya2,cv2.COLOR_RGB2LAB)
    l,a,b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=5.0,tileGridSize=((8,8)))
    cl = clahe.apply(l)
    limg = cv2.merge((cl,a,b))
    son = cv2.cvtColor(limg,cv2.COLOR_LAB2RGB)
    med_son = cv2.medianBlur(son,3)
    arka_plan = cv2.medianBlur(son,37)
    maske = cv2.addWeighted(med_son,1,arka_plan,-1,255)
    son_img = cv2.bitwise_and(maske,med_son)
    return son_img


In [ ]:
#read images, do division, save results batches
path = 'DR-dataset/train/'
numberTrainImagesPerClass = 600
numberTestImagesPerClass = 100
size_batch = 255
sizeOfOriginalImage = 1024
numberOfClass = 5

def createBatches(path,numberTrainImagesPerClass,numberTestImagesPerClass,size_batch,sizeOfOriginalImage,numberOfClass):
    numberOfBatch = sizeOfOriginalImage // size_batch
    trainX = np.array([3000,16,255,255,3], np.int8)
    trainY = np.array([3000,1])
    l = 0
    for c in range(numberOfClass):
      # the path of each class
      path_class = path + str(c)
      # get the list of images in each class
      list_images =  os.listdir(path_class)
      # choose randomly 300 images from the previous list
      choosed_images = random.sample(list_images, numberTrainImagesPerClass+numberTestImagesPerClass)
      # get train dataset
      for k in range(numberTrainImagesPerClass):
        # get the path of each image
        image = choosed_images[k]
        path_image = path_class+'/'+ image
        # read the image
        img = cv2.imread(path_image)
        try:
          img = preprocessFunction(img)
        except:
          pass
        img = cv2.resize(img, dsize=(sizeOfOriginalImage, sizeOfOriginalImage))
        numeroBatch = 1
        m = 0
        for i in range(numberOfBatch):
          for j in range(numberOfBatch):
            batch = img[i*size_batch:i*size_batch+size_batch,j*size_batch:j*size_batch+size_batch,:]
            try:
                trainX[l][m] = batch
            except Exception as e:
                print('error_train'+str(e))
            m = m+1
            numeroBatch = numeroBatch + 1
        trainY[l] = c
        l = l+1
createBatches(path,numberTrainImagesPerClass,numberTestImagesPerClass,size_batch,sizeOfOriginalImage,numberOfClass)

error_train'numpy.int8' object does not support item assignment
error_train'numpy.int8' object does not support item assignment
error_train'numpy.int8' object does not support item assignment
error_train'numpy.int8' object does not support item assignment
error_train'numpy.int8' object does not support item assignment
error_train'numpy.int8' object does not support item assignment
error_train'numpy.int8' object does not support item assignment
error_train'numpy.int8' object does not support item assignment
error_train'numpy.int8' object does not support item assignment
error_train'numpy.int8' object does not support item assignment
error_train'numpy.int8' object does not support item assignment
error_train'numpy.int8' object does not support item assignment
error_train'numpy.int8' object does not support item assignment
error_train'numpy.int8' object does not support item assignment
error_train'numpy.int8' object does not support item assignment
error_train'numpy.int8' object does not 

IndexError: ignored

In [ ]:
# creating the batches folder
path_batch_train = 'batchesTrain'
path_batch_test = 'batchesTest'

def createBatchFolder(pathName):
    try:
        os.mkdir(pathName)
    except OSError:
        print ("Creation of the directory %s failed" % pathName)
    else:
        print ("Successfully created the directory %s " % pathName)
    finally:
        for i in range(1,17):
          try:
              pathBatch = pathName+'/'+str(i)
              os.mkdir(pathBatch)
          except OSError:
              print ("Creation of the directory %s failed" % pathName)
          finally:
            for j in range(5):
              try:
                  pathClass = pathBatch+'/'+ str(j)
                  os.mkdir(pathClass)
              except OSError:
                  print ("Creation of the directory %s failed" % pathName)
# call the create path function
createBatchFolder(path_batch_train)
createBatchFolder(path_batch_test)

In [ ]:
#read images, do division, save results batches
path = 'DR-dataset/train/'
numberTrainImagesPerClass = 600
numberTestImagesPerClass = 100
size_batch = 255
sizeOfOriginalImage = 1024
numberOfClass = 5

def createBatches(path,path_batch,path_test,numberTrainImagesPerClass,numberTestImagesPerClass,size_batch,sizeOfOriginalImage,numberOfClass):
    numberOfBatch = sizeOfOriginalImage // size_batch
    for c in range(numberOfClass):
      # the path of each class
      path_class = path + str(c)
      # get the list of images in each class
      list_images =  os.listdir(path_class)
      # choose randomly 300 images from the previous list
      choosed_images = random.sample(list_images, numberTrainImagesPerClass+numberTestImagesPerClass)
      # get train dataset
      for k in range(numberTrainImagesPerClass):
        # get the path of each image
        image = choosed_images[k]
        path_image = path_class+'/'+ image
        # read the image
        img = cv2.imread(path_image)
        try:
          img = preprocessFunction(img)
        except:
          pass
        img = cv2.resize(img, dsize=(sizeOfOriginalImage, sizeOfOriginalImage))
        numeroBatch = 1
        for i in range(numberOfBatch):
          for j in range(numberOfBatch):
            batch = img[i*size_batch:i*size_batch+size_batch,j*size_batch:j*size_batch+size_batch,:]
            try:
                newPath = path_batch+'/'+str(numeroBatch)+'/'+str(c)+'/'+image
                cv2.imwrite(newPath,batch)
            except Exception as e:
                print(img.shape)
                print('error_train'+str(e))
            numeroBatch = numeroBatch + 1
      # get the test dataset
      for k in range(numberTrainImagesPerClass,numberTrainImagesPerClass+numberTestImagesPerClass):
        # get the path of each image
        image = choosed_images[k]
        path_image = path_class+'/'+image
        # read the image
        img = cv2.imread(path_image)
        img = preprocessFunction(img)
        img = cv2.resize(img, dsize=(sizeOfOriginalImage, sizeOfOriginalImage))
        numeroBatch = 1
        for i in range(numberOfBatch):
          for j in range(numberOfBatch):
            batch = img[i*size_batch:i*size_batch+size_batch,j*size_batch:j*size_batch+size_batch,:]
            try:
                newPath = path_test+'/'+str(numeroBatch)+'/'+str(c)+'/'+image
                cv2.imwrite(newPath,batch)
            except Exception as e: 
                print('error_test'+str(e))
            numeroBatch = numeroBatch + 1

createBatches(path,path_batch_train,path_batch_test,numberTrainImagesPerClass,numberTestImagesPerClass,size_batch,sizeOfOriginalImage,numberOfClass)

In [ ]:
# augmenttaion
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib
import os
import cv2
%matplotlib inline

for i in range(1,17):
  for j in range(5):
    BASE_DIR = "batchesTrain/"+str(i)+'/'+str(j)
    NEW_DIR= "batchesTrain/"+str(i)+'/'+str(j)
    list_dir =  os.listdir(BASE_DIR)
    from google.colab.patches import cv2_imshow
    for images in list_dir:
        image = cv2.imread(BASE_DIR + "/"+ images)
        #image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
            
        #rotation
        rotate=iaa.Affine(rotate=(0, 30))
        rotated_image=rotate.augment_image(image)
        cv2.imwrite(NEW_DIR + "/" + 'rotated-'+images,rotated_image)

        #zoom
        crop = iaa.Crop(percent=(0.07, 0.12)) # crop image
        corp_image=crop.augment_image(image)
        cv2.imwrite(NEW_DIR + "/" + 'zoomed-'+images,corp_image)

        #shear
        crop = iaa.Affine(shear=10)
        corp_image=crop.augment_image(image)
        cv2.imwrite(NEW_DIR + "/" + 'affine-'+images,corp_image)

        #flip H
        flip_hr=iaa.Fliplr(p=1.0)
        flip_hr_image= flip_hr.augment_image(image)
        cv2.imwrite(NEW_DIR + "/" + 'fliph-'+images,flip_hr_image)


# create the Multi Channel Model


In [ ]:
# create one channel
def densenet(input_shape, filters = 32):  
    #batch norm + relu + conv
    def bn_rl_conv(x,filters,kernel=1,strides=1):
        
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(filters, kernel, strides=strides,padding = 'same')(x)
        return x
    
    def dense_block(x, repetition):
        
        for _ in range(repetition):
            y = bn_rl_conv(x, 4*filters)
            y = bn_rl_conv(y, filters, 3)
            x = concatenate([y,x])
        return x
        
    def transition_layer(x):
        
        x = bn_rl_conv(x, K.int_shape(x)[-1] //2 )
        x = AvgPool2D(2, strides = 2, padding = 'same')(x)
        return x
    
    input = Input (input_shape)
    x = Conv2D(64, 7, strides = 2, padding = 'same')(input)
    x = MaxPool2D(3, strides = 2, padding = 'same')(x)
    
    for repetition in [6,12,24,16]:
        
        d = dense_block(x, repetition)
        x = transition_layer(d)
    x = GlobalAveragePooling2D()(d)
    flat = Flatten()(x)
    return flat,input

  

# define the model

channels = []
inputs = []
for i in range(16):
  channel,input =  densenet((224,224,3))
  channels.append(channel)
  inputs.append(input)
# merge
merged = concatenate([channels[0], channels[1], channels[2],channels[3], channels[4],channels[5],
                      channels[6], channels[7], channels[8],channels[9], channels[10],
                      channels[11],channels[12], channels[13], channels[14], channels[15]])

outputs = Dense(5, activation='softmax')(merged)
model = Model(inputs=inputs, outputs=outputs)
# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.Adam(lr=0.00005),
    metrics=['accuracy']
)
# summarize
#print(model.summary())
#plot_model(model, show_shapes=True, to_file='multichannel.png')

#Read data as Image Data Generator Object

In [ ]:
path_batch = 'batchesTrain'
train_generators = []
val_generators = []

for i in range(1,17):
  batchSize = 32
  pathBatch = path_batch+'/'+str(i)+'/'
  # ******** for train dataset

  train_datagen = ImageDataGenerator(
      rescale=1./255,
      validation_split=0.15
      ) 

  train_generator = train_datagen.flow_from_directory(
      pathBatch,
      classes = ['0','1','2','3','4'],
      target_size=(224, 224),
      color_mode='rgb',
      batch_size=batchSize,
      class_mode='categorical',
      subset='training',
      shuffle=True)

  # ******* for validation dataset
  val_generator = train_datagen.flow_from_directory(
      pathBatch,
      classes = ['0','1','2','3','4'],
      class_mode='categorical',
      target_size=(224, 224),
      color_mode='rgb',
      subset='validation'
  )
  train_generators.append(train_generator)
  val_generators.append(train_generator)


# start the training

In [ ]:
earlystopper = EarlyStopping(
            monitor = "val_loss",
            mode="min",
            patience=10,
            verbose=1
  )
model_checkpoint = ModelCheckpoint(
              'drive/MyDrive/models/task-c/modelBatch'+str(i)+'.h5',
              monitor='val_loss',
              verbose=1,
              save_best_only=True
  )
model.compile(optimizer=optimizers.Adam(lr=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  


In [ ]:
model.fit(list_batches, trainY, epochs=15, batch_size=4,validation_data=(list_batches_test,trainY_test),verbose=1)

Epoch 1/15
377/377 [==============================] - 383s 1s/step - loss: 1.6114 - accuracy: 0.2930 - val_loss: 1.9128 - val_accuracy: 0.2260
Epoch 2/15
377/377 [==============================] - 341s 905ms/step - loss: 1.4925 - accuracy: 0.3528 - val_loss: 2.0004 - val_accuracy: 0.2720
Epoch 3/15
377/377 [==============================] - 341s 904ms/step - loss: 1.4440 - accuracy: 0.3960 - val_loss: 2.0699 - val_accuracy: 0.2800
Epoch 4/15
377/377 [==============================] - 340s 902ms/step - loss: 1.3788 - accuracy: 0.4279 - val_loss: 1.9888 - val_accuracy: 0.2860
Epoch 5/15
377/377 [==============================] - 339s 899ms/step - loss: 1.3842 - accuracy: 0.4146 - val_loss: 1.9450 - val_accuracy: 0.2740
Epoch 6/15
377/377 [==============================] - 339s 900ms/step - loss: 1.3085 - accuracy: 0.4598 - val_loss: 1.9543 - val_accuracy: 0.2980
Epoch 7/15
377/377 [==============================] - 339s 900ms/step - loss: 1.2697 - accuracy: 0.5010 - val_loss: 2.4693 - va

# drawing results

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
#plt.plot(epochs, loss, 'g', label='Training loss')
#plt.plot(epochs, val_loss, 'y', label='Validation loss')
plt.title('Training and validation')
plt.legend(loc=0)
plt.figure()
plt.savefig('results/batch'+str(i)+'.png')
plt.show()

# evaluate using validation data